Start a `ipcluster` from the Cluster tab in Jupyter or use the command:

```ipcluster start``` 

in a terminal.

In [ ]:
# import os
# from scripts.hpc05 import HPC05Client
# os.environ['SSH_AUTH_SOCK'] = os.path.join(os.path.expanduser('~'), 'ssh-agent.socket')
# cluster = HPC05Client()

In [ ]:
from ipyparallel import Client
cluster = Client()
v = cluster[:]
lview = cluster.load_balanced_view()
len(v)

# Make sure to add the correct path like:

    sys.path.append("/path/where/to/ipynb/runs")
    
The code currently assumes the scripts live in `~/orbitalfield/`.

In [ ]:
%%px --local
import numpy as np
import sys
import os
sys.path.append(os.path.join(os.path.expanduser('~'), 'orbitalfield'))
from fun import *
from fun2 import *
def evs(lead, p):
    h0 = lead.cell_hamiltonian(args=[p])
    t0 = lead.inter_cell_hopping(args=[p])
    ham = h0 + t0 + t0.conj().T
    ev = np.linalg.eigvalsh(ham)
    return np.abs(ev).min()

In [ ]:
import holoviews as hv
from itertools import product
hv.notebook_extension()
%opts Contours (linewidth=1 color='k') [aspect='square']
%output size=100 dpi=100

Uncomment the lines for the wire that you want to use.

In [ ]:
%%px --local 
constants.a = 10
p = make_params(t_interface=7/8*constants.t, Delta=68.4, r1=50, r2=70, 
                phi=135, orbital=True, A_correction=False) #r2=70
p.V = lambda x, y, z: 2 / 50 * z

# p.angle = 0 # WIRE WITH SC ON TOP
p.angle = 45 # WIRE WITH SC ON SIDE
lead = make_3d_wire_external_sc(a=constants.a, r1=p.r1, r2=p.r2, angle=p.angle, phi=p.phi)

You can specify the intervals of $\mu$ and $\Delta$ that you want to check.

In [ ]:
mus = np.linspace(0, 20, 100)
deltas = np.linspace(60, 80, 50)
vals = list(product(deltas, mus))
v.scatter('vals', vals)
%px res = [evs(lead, p) for p.Delta, p.mu in vals]
res = np.reshape(v.gather('res', block=True), (len(deltas), len(mus)))

Run the next cell and use the slider to find the correct $\Delta$.

In [ ]:
Delta_ind = {delta: hv.Contours((mus, evs), kdims=[dimensions.mu, dimensions.delta_ind]) 
      for evs, delta in zip(res, deltas)}

hm = hv.HoloMap(Delta_ind, kdims=[dimensions.delta])
hm.select(mu=(0, 20), **{'$\Delta_{ind}$': (0.2, 10)}) * hv.HLine(0.25)

As you can see $\Delta=68.4$ meV is the parameter we need with the superconductor on the side if we want $\Delta_{ind}$.

In [ ]:
delta = 68.4
chosen_curve = Delta_ind[nearest(deltas, delta)].select(mu=(4, 6), **{'$\Delta_{ind}$': (0.23, 0.28)})
(chosen_curve * hv.HLine(0.25)).relabel(r'$\Delta={}$ meV, $t_s=7/8t$'.format(delta))